In [1193]:
import re
from collections import Counter, defaultdict
from functools import cache
from heapq import heappop, heappush
from itertools import combinations
from math import inf, lcm, prod, sqrt

import black
import jupyter_black
import numpy as np

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2023/{day}-example.txt" if example else f"2023/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [94]:
# Day 1: Trebuchet?!
numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}


def calibration_value(line, part1=True):
    numbers_found = ""
    for start, char in enumerate(line):
        if char.isdigit():
            numbers_found += char
        if part1:
            continue
        for key, value in numbers.items():
            if line.startswith(key, start):
                numbers_found += value
    calibration_value = numbers_found[0] + numbers_found[-1]
    return int(calibration_value)


games = data(1)
points = sum([calibration_value(line) for line in games])
print(f"Part 1: {points}")  # 53921
points = sum([calibration_value(line, part1=False) for line in games])
print(f"Part 2: {points}")  # 54676

Part 1: 53921
Part 2: 54676


In [151]:
# Day 2: Cube Conundrum
games = data(2)
possible_games = 0
powers = []
for game_nr, game in enumerate(games, 1):
    max_cubes_seen = {"red": 0, "blue": 0, "green": 0}
    cubes_seen = game.split(": ")[1].split("; ")
    for cubes in cubes_seen:
        for cube in cubes.split(", "):
            count, color = cube.split()
            count = int(count)
            if count > max_cubes_seen[color]:
                max_cubes_seen[color] = count
    if (
        max_cubes_seen["red"] <= 12
        and max_cubes_seen["green"] <= 13
        and max_cubes_seen["blue"] <= 14
    ):
        possible_games += game_nr
    powers.append(prod(max_cubes_seen.values()))

print(f"Part 1: {possible_games}")  # 2563
print(f"Part 2: {sum(powers)}")  # 70768

Part 1: 2563
Part 2: 70768


In [181]:
# Day 3: Gear Ratios
def neighbors(point):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for (r, c) in potential:
        if 0 <= r < len(expanded_rows[0]) and 0 <= c < len(expanded_rows):
            yield (r, c)


expanded_rows = data(3)
gears = defaultdict(list)
part_numbers = []

for row_number, row in enumerate(expanded_rows):
    current_number = ""
    to_check = set()
    # Add '.' to catch numbers at end of line and avoid separate check after loop
    # completion
    for col_number, char in enumerate(row + "."):
        if char.isdigit():
            current_number += char
            for point in neighbors((row_number, col_number)):
                to_check.add(point)
        elif current_number:
            for (r, c) in to_check:
                if expanded_rows[r][c] not in "0123456789.":
                    # Symbol found, we are a part_number
                    part_numbers.append(int(current_number))
                if expanded_rows[r][c] == "*":
                    # Gear found, add number connected to this gear
                    gears[(r, c)].append(int(current_number))
            current_number = ""
            to_check = set()
print(f"Part 1: {sum(part_numbers)}")  # 514969
part2 = sum((prod(gear) for gear in gears.values() if len(gear) == 2))
print(f"Part 2: {part2}")  # 78915902

Part 1: 514969
Part 2: 78915902


In [94]:
# Day 4: Scratchcards
cards = data(4)
points = 0
total_cards = Counter([*range(len(cards))])  # One of each card to begin with
for card_nr, card in enumerate(cards):
    winning_numbers, my_numbers = map(ints, card.split(": ")[1].split(" | "))
    winners = sum(1 for number in my_numbers if number in winning_numbers)
    if winners:
        points += 2 ** (winners - 1)
        total_cards.update(
            total_cards[card_nr] * [*range(card_nr + 1, card_nr + winners + 1)]
        )

print(f"Part 1: {points}")  # 28750
print(f"Part 2: {sum(total_cards.values())}")  # 10212704

Part 1: 28750
Part 2: 10212704


In [17]:
# Day 5: If You Give A Seed A Fertilizer
def get_destination(seed, mapping):
    for dest, source, range_ in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        if source <= seed <= source + range_:
            return dest - source + seed
    return seed


def get_location(seed, mappings):
    for mapping in mappings:
        seed = get_destination(seed, mapping)
    return seed


patterns = data(5, sep="\n\n")
seeds = ints(patterns[0])
mappings = [tuple(ints(chunk)) for chunk in patterns[1:]]

print(f"Part 1: {min(get_location(seed, mappings) for seed in seeds)}")

# -- Part 2 -- Yucky.
def new_ranges(low, high, dest, source, range):
    """Return two lists (lows and highs) based on a continuous range from low to high and
    a mapping and then two lists including remaining low-high pairs to check (one list
    includes lows, one includes highs)"""
    offset = dest - source
    new_low = source
    new_high = source + range - 1
    # print(f"{new_low=} {new_high=}")
    if low >= new_low and high <= new_high:
        # Range intact, return low + offset, high + offset
        # print(f"Complete overlap {offset=}")
        return [low + offset], [high + offset], [], []
    if low <= source and new_low <= high <= new_high:
        # -------low---------------------high------------
        #         |                       |
        #             |------offset-------|
        #             |                         |
        # ---------new_low-----------------new_high------
        # print(f"Partial overlap, case 1 {offset=}")
        return (
            [low, new_low + offset],
            [new_low - 1, high + offset],
            [low],
            [new_low - 1],
        )
    if low < new_low and high > new_high:
        # -------low----------------------------high-------
        #         |                              |
        #              |------offset-------|
        #              |                   |
        # ----------new_low------------new_high------------
        # print(f"Partial overlap, case 2 {offset=}")
        return (
            [low, new_low + offset, new_high + 1],
            [
                new_low - 1,
                new_high + offset,
                high,
            ],
            [low, new_high + 1],
            [new_low - 1, high],
        )
    if new_low <= low <= new_high and high > new_high:
        # -------low---------------------high-----------
        #         |                       |
        #         |------offset-------|
        #      |                      |
        # ---new_low--------------new_high--------------
        # print(f"Partial overlap, case 3 {offset=}")
        return (
            [low + offset, new_high + 1],
            [
                new_high + offset,
                high,
            ],
            [new_high + 1],
            [high],
        )
    # Non overlapping
    # print(f"No overlap")
    return [], [], [], []


def apply_mapping(low, high, mapping):
    new_lows, new_highs = [], []
    # print(f"    {low=}     {high=}")
    for dest, source, range in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        lows_remaining = [low]
        highs_remaining = [high]
        while len(lows_remaining) and len(highs_remaining):
            l, h, lows_remaining, highs_remaining = new_ranges(
                lows_remaining[0], highs_remaining[0], dest, source, range
            )
            new_lows += l
            new_highs += h
            # print(lows_remaining, highs_remaining)
    if not new_lows:
        new_lows = [low]
    if not new_highs:
        new_highs = [high]
    # print(f"{new_lows=} {new_highs=}")
    return new_lows, new_highs


ranges = ints(patterns[0])
mappings = [ints(chunk) for chunk in patterns[1:]]
input_lows = ranges[0::2]
input_highs = [low + length - 1 for low, length in zip(input_lows, ranges[1::2])]

locations = []
for lows, highs in zip(input_lows, input_highs):
    lows = [lows]
    highs = [highs]
    for mapping in mappings:
        # print("--------New mapping---------")
        # print(f"{mapping=}")
        new_lows, new_highs = [], []
        for low, high in zip(lows, highs):
            l, h = apply_mapping(low, high, mapping)
            new_lows += l
            new_highs += h
        lows = new_lows
        highs = new_highs
    locations.append(min(lows))

print(locations)

# 2520479 is the right answer, but by god this was an awful hack. The zeroes for the last
# ranges doesn't make sense to me, but I'm tired of this day now.

Part 1: 462648396
[4111139999, 942937748, 517161737, 3475769331, 2520479, 450060781, 56747007, 0, 0, 0]


In [217]:
# Day 6: Wait For It
times, distances = data(6, ints)

# Travel distance = speed * (time - button_press), where speed = button_press
part1 = prod(
    sum(
        1
        for button_press in range(time)
        if button_press * (time - button_press) > distance
    )
    for time, distance in zip(times, distances)
)
print(f"Part 1: {part1}")  # 512295

time = int("".join(map(str, times)))
expansion = int("".join(map(str, distances)))
part2 = sum(
    1 for button_press in range(time) if button_press * (time - button_press) > expansion
)
print(f"Part 2: {part2}")  # 36530883 (6s)

Part 1: 512295
Part 2: 36530883


In [278]:
# Day 7: Camel Cards
def most_common_character(s):
    s = sorted(s, reverse=True)
    freq = Counter(s)
    # Sort the characters by frequency
    sorted_chars = sorted(freq, key=freq.get, reverse=True)
    return sorted_chars[0]


lines = data(7)

hands = []
for first_part in lines:
    text, bid = first_part.split()
    # Mapping A, K, Q, J, T -> e, d, c, b, a
    hand = text.translate(str.maketrans("AKQJT", "edcba"))
    if hand == "bbbbb":
        pretend_hand = "22222"
    else:
        # replace all Jokers ('b') with the most common character *excluding* b
        pretend_hand = hand.replace("b", most_common_character(hand.replace("b", "")))
    hands.append((pretend_hand, hand, int(bid)))

# Part 1
decorated = [
    (
        sorted(Counter(hand).values(), reverse=True),
        hand,
        bid,
    )
    for _, hand, bid in hands
]
decorated.sort()
print(f"Part 1: {sum(rank * item[-1] for rank, item in enumerate(decorated, 1))}")

# Part 2
decorated = [
    (
        sorted(Counter(pretend_hand).values(), reverse=True),
        hand.replace("b", "1"),  # Jokers are worth the least
        bid,
    )
    for pretend_hand, hand, bid in hands
]
decorated.sort()
# 244848487
print(f"Part 2: {sum(rank * item[-1] for rank, item in enumerate(decorated, 1))}")

Part 1: 246409899
Part 2: 244848487


In [47]:
# Day 8: Haunted Wasteland
patterns = data(8, sep="\n\n")
instructions = [int(step.replace("R", "1").replace("L", "0")) for step in patterns[0]]
hops = {
    line.split(" = ")[0]: line.split(" = ")[1].strip("()").split(", ")
    for line in patterns[1].splitlines()
}

# Part 1
steps = 0
node = "AAA"
while node != "ZZZ":
    node = hops[node][instructions[steps % len(instructions)]]
    steps += 1
print(f"Part 1: {steps}")  # 19667

# Part 2
nodes = [start for start in hops if start[-1] == "A"]
# Find the frequencies for the different starting points ending with 'A'
freqs = []
steps = 0
while len(freqs) < len(nodes):
    for pos, node in enumerate(nodes):
        next = hops[node][instructions[steps % len(instructions)]]
        if next[-1] == "Z":
            freqs.append(steps + 1)
        nodes[pos] = next
    steps += 1
# Least Common Multiple is when all the frequencies match up
print(f"Part 2: {lcm(*freqs)}")  # 19185263738117

Part 1: 19667
Part 2: 19185263738117


In [91]:
# Day 9: Mirage Maintenance
def differences(numbers):
    return [b - a for a, b in zip(numbers, numbers[1:])]


def predict(value):
    prediction = [value]
    while any(prediction[-1]):
        prediction.append(differences(prediction[-1]))
    return sum(value[-1] for value in prediction)


def predict2(value):
    prediction = [value]
    while any(prediction[-1]):
        prediction.append(differences(prediction[-1]))
    temp = [value[0] for value in prediction[::-1]]
    # value1line1-value0line1 = value0line0
    # value0line1 = value1line1-value0line0 # a = b - c
    c = 0
    for b in temp:
        a = b - c
        c = a
    return a


sequence = data(9, ints)
print(f"Part 1: {sum(predict(value) for value in sequence)}")  # 1834108701
print(f"Part 2: {sum(predict2(value) for value in sequence)}")  # 993

Part 1: 1834108701
Part 2: 993


In [94]:
# Day 9: Mirage Maintenance
def differences(numbers):
    return [b - a for a, b in zip(numbers, numbers[1:])]


def predict(value):
    result = [value]
    while any(result[-1]):
        result.append(differences(result[-1]))
    return result


def part1(prediction):
    return sum(value[-1] for value in prediction)


def part2(prediction):
    # See problem description for details
    # A B X1 X2 X3
    #  C Y1 Y2 Y3
    # A = B - C
    c = 0
    for b in prediction[::-1]:
        a = b[0] - c
        c = a
    return a


sequence = data(9, ints)
print(f"Part 1: {sum(part1(predict(value)) for value in sequence)}")  # 1834108701
print(f"Part 2: {sum(part2(predict(value)) for value in sequence)}")  # 993

Part 1: 1834108701
Part 2: 993


In [104]:
# Day 10: Pipe Maze
def new_dir(position, dir):
    right_turn = -1j
    left_turn = 1j
    # char, dir: turn
    # [-1, 1j, 1, -1j] = N, E, S, W
    turns = {
        ("F", -1): right_turn,
        ("F", -1j): left_turn,
        ("J", 1): right_turn,
        ("J", 1j): left_turn,
        ("7", 1j): right_turn,
        ("7", -1): left_turn,
        ("L", 1): left_turn,
        ("L", -1j): right_turn,
    }
    if (expanded_rows[position], dir) in turns:
        return dir * turns[(expanded_rows[position], dir)]
    return dir


def create_polygon(start_position, dir=-1):
    # Starting dir (-1 == North) comes from looking manually at the map
    position = start_position
    polygon = {start_position}
    while position != start_position or len(polygon) < 2:
        dir = new_dir(position, dir)
        position = position + dir
        polygon.add(position)

    return {(int(p.real), int(p.imag)) for p in polygon}


def create_intersections(polygon):
    "Create a grid with the number of intersections for each point based on a straight line from left to right. See https://en.wikipedia.org/wiki/Point_in_polygon"
    intersections = {}
    for row in range(max(polygon)[0]):
        current = 0
        last_bend = ""
        for col in range(max(polygon, key=lambda x: x[1])[1]):
            pos = row + 1j * col
            if (row, col) in polygon:
                if expanded_rows[pos] == "|":
                    current += 1
                # 7---L and J---F are equivalent to a straight vertical line
                if expanded_rows[pos] == "7" and last_bend == "L":
                    current += 1
                if expanded_rows[pos] == "J" and last_bend == "F":
                    current += 1
                if expanded_rows[pos] in "FJL7":
                    last_bend = expanded_rows[pos]
            else:
                intersections[(row, col)] = current
    return intersections


def points_inside_polygon(intersections):
    return sum(1 for p in intersections.values() if p % 2)


lines = data(10)
expanded_rows = {}  # (row, col): char
start_position = 0
for row, first_part in enumerate(lines):
    for col, char in enumerate(first_part):
        expanded_rows[(row + 1j * col)] = char
        if char == "S":
            start_position = row + 1j * col
            expanded_rows[(row + 1j * col)] = "J"  # Manually looking at map

polygon = create_polygon(start_position)
print(f"Part 1: {len(polygon) // 2}")  # 6733
print(f"Part 2: {points_inside_polygon(create_intersections(polygon))}")  # 435

Part 1: 6733
Part 2: 435


In [15]:
# Day 11: Cosmic Expansion
def create_grid(lines):
    grid = {
        (row, col): char
        for row, line in enumerate(lines)
        for col, char in enumerate(line)
    }
    return grid


def shortest_paths(start, expansion):
    """
    Calculate shortest path from start to all possible positions
    """
    frontier = []
    heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = defaultdict(lambda: inf)
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next, expansion)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                heappush(frontier, (priority, next))
                came_from[next] = current

    return cost_so_far


def cost(current, next, expansion):
    row, col = current
    to_row, to_col = next
    # Vertical movement
    if row - to_row and row_empty(row):
        return expansion
    # Horizontal movement
    if col - to_col and col_empty(col):
        return expansion
    return 1


@cache
def row_empty(row):
    cols = max(create_grid.keys(), key=lambda x: x[1])[1]
    return all(create_grid[(row, c)] == "." for c in range(cols + 1))


@cache
def col_empty(col):
    rows = max(create_grid.keys())[0]
    return all(create_grid[(r, col)] == "." for r in range(rows + 1))


@cache
def moves(current):
    row, col = current
    candidates = {(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)}
    return {move for move in candidates if move in create_grid.keys()}


def total_cost(expansion=2):
    galaxies = {coord for coord, char in create_grid.items() if char == "#"}
    pairs = set(combinations(galaxies, 2))
    total_cost = 0
    for galaxy in galaxies:
        cost_to_reach = shortest_paths(galaxy, expansion)
        for (from_, to_) in pairs.copy():
            if galaxy == from_:
                total_cost += cost_to_reach[to_]
                pairs.remove((from_, to_))
    return total_cost


lines = data(11)
create_grid = create_grid(lines)
print(f"Part 1: {total_cost(expansion=2)}")  # 10165598, 25s
print(f"Part 2: {total_cost(expansion=1_000_000)}")  # 678728808158, 35s

Part 1: 10165598
Part 2: 678728808158


In [525]:
# Day 12: Hot Springs
@cache
def arrangements(springs, damaged):
    # Initial input springs need to have a '.' at the end to make code below easier
    # Base cases
    if not damaged:
        if all(c in "?." for c in springs):
            return 1
        return 0
    if not springs:
        return 0

    damage = damaged[0]
    spring = springs[0]
    if spring == "?":
        if all(c in "?#" for c in springs[:damage]) and springs[damage] in "?.":
            # If we can get a full chunk of #:s and a stop, return that as one possibility
            # + the case where ? is a '.' as the other possibility
            return arrangements(springs[damage + 1 :], damaged[1:]) + arrangements(
                springs[1:], damaged
            )
        else:
            # We can't get a full chunk of '#', just return the case where ? is a '.'
            return arrangements(springs[1:], damaged)
    if spring == "#":
        # Need complete chunk + a pause
        if all(c in "?#" for c in springs[:damage]) and springs[damage] in "?.":
            return arrangements(springs[damage + 1 :], damaged[1:])
        else:
            return 0
    if spring == ".":
        return arrangements(springs[1:], damaged)


def unfold(springs, numbers):
    "Unfold springs and numbers, and add '.' to unfolded springs"
    return (springs + "?") * 4 + springs + ".", numbers * 5


lines = data(12)
part1, part2 = 0, 0
for first_part in lines:
    springs, damages = first_part.split()
    damages = tuple(ints(damages))
    part1 += arrangements(springs + ".", damages)
    part2 += arrangements(*unfold(springs, damages))

print(f"Part 1: {part1}")  # 7771
print(f"Part 2: {part2}")  # 10861030975833

Part 1: 7935
Part 2: 10887924681597


In [720]:
# Day 13: Point of Incidence
def rows(grid):
    return max(grid, key=lambda coord: coord[0])[0] + 1


def cols(grid):
    return max(grid, key=lambda coord: coord[1])[1] + 1


def create_grid(chunk):
    return {
        (r, c): char
        for r, line in enumerate(chunk.splitlines())
        for c, char in enumerate(line)
    }


def first_item(items):
    if len(items):
        return first(items)
    else:
        return 0


def cols_to_the_left(grid):
    def row_is_mirrored(grid, row, mirror_column):
        first_part = [grid[(row, col)] for col in range(mirror_column)]
        second_part = [grid[(row, col)] for col in range(mirror_column, cols(grid))]
        return all(a == b for a, b in zip(first_part[::-1], second_part))

    def mirror_columns(grid, row):
        return {col for col in range(1, cols(grid)) if row_is_mirrored(grid, row, col)}

    set_list = [mirror_columns(grid, row) for row in range(rows(grid))]
    common_items = set.intersection(*set_list)
    return common_items


def rows_above(grid):
    def col_is_mirrored(grid, col, mirror_row):
        first_part = [grid[(row, col)] for row in range(mirror_row)]
        second_part = [grid[(row, col)] for row in range(mirror_row, rows(grid))]
        return all(a == b for a, b in zip(first_part[::-1], second_part))

    def mirror_rows(grid, col):
        return {row for row in range(1, rows(grid)) if col_is_mirrored(grid, col, row)}

    set_list = [mirror_rows(grid, col) for col in range(cols(grid))]
    common_items = set.intersection(*set_list)
    return common_items


def smudge(grid, row, col):
    "Modifies grid in place, make sure to restore"
    smudge = {"#": ".", ".": "#"}
    grid[(row, col)] = smudge[grid[(row, col)]]


def smudged_lines(grid, v, h):
    "Returns new values for vertical, horizontal"
    for row in range(rows(grid)):
        for col in range(cols(grid)):
            smudge(grid, row, col)
            if v_smudged := cols_to_the_left(grid):
                if v_smudged != v:
                    smudge(grid, row, col)
                    return v_smudged - v, set()

            if h_smudged := rows_above(grid):
                if h_smudged != h:
                    smudge(grid, row, col)
                    return set(), h_smudged - h
            smudge(grid, row, col)


patterns = data(13, sep="\n\n")
part1, part2 = 0, 0
for pattern in patterns:
    grid = create_grid(pattern)
    v = cols_to_the_left(grid)
    h = rows_above(grid)
    new_v, new_h = smudged_lines(grid, v, h)
    part1 += first_item(v) + first_item(h) * 100
    part2 += first_item(new_v) + first_item(new_h) * 100

print(f"Part 1: {part1}")  # 43614
print(f"Part 2: {part2}")  # 36771, 45s for both parts

Part 1: 43614
Part 2: 36771


In [913]:
# Day 14: Parabolic Reflector Dish, optimized code with numpy
def create_grid(chunk):
    return np.array([list(line) for line in chunk.splitlines()], dtype=str)


def tilt_platform_east(platform):
    def tilt_line_east(line):
        # Move all O:s right up until the next #
        # Will add an extra '#' at the end, chop that off with [:-1]
        return (
            "".join(
                part.count(".") * "." + part.count("O") * "O" + "#"
                for part in line.split("#")
            )[:-1]
            + "\n"
        )

    result = ""
    for line in platform:
        line = "".join(line)
        result += tilt_line_east(line)
    return create_grid(result)


def spin_cycle(platform):
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # North tilt
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # West tilt
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # South tilt
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # East tilt

    return platform


def load(platform):
    result = 0
    for weight, row in enumerate(platform):
        result += (platform.shape[0] - weight) * np.count_nonzero(row == "O")
    return result


def load_at_iteration(start_of_sequence, sequence, iteration=1000000000):
    pos = (iteration - start_of_sequence) % len(sequence)
    return sequence[pos]


chunk = open("2023/14.txt").read().rstrip()

platform = create_grid(chunk)
platform = np.rot90(platform, -1)
platform = tilt_platform_east(platform)
print(f"Part 1: {load(np.rot90(platform, 1))}")  # 110565

platform = create_grid(chunk)
# Pattern emerges starting at cycle 102 and lasts until 143, then 144-185 is the same
# numbers repeated, etc. See day14-output.txt for output.
# for i in range(1, 144):
#     platform = spin_cycle(platform)
#     print(f"{i:>4.0f}: {load(platform)}")

# Here is the observed pattern:
# fmt: off
sequence = [ 89803, 89824, 89836, 89855, 89878, 89887, 89896, 89900, 89880, 89873, 89849, 89813, 89800, 89795, 89799, 89819, 89845, 89851, 89873, 89896, 89892, 89895, 89889, 89869, 89844, 89822, 89796, 89790, 89808, 89815, 89840, 89860, 89869, 89891, 89901, 89891, 89884, 89878, 89840, 89817, 89805, 89786 ]
# fmt: on
# And observed start of pattern:
start = 102

print(f"Part 2: {load_at_iteration(start, sequence, 1000000000)}")  # 89845

Part 1: 110565
Part 2: 89845


In [999]:
# Day 15: Lens Library
def hash(value, char):
    return (value + ord(char)) * 17 % 256


def hash_string(string):
    value = 0
    for char in string:
        value = hash(value, char)
    return value


def box_index(box, label):
    for i, item in enumerate(box):
        if label == item[0]:
            return i
    return -1


def boxes(lenses):
    boxes = defaultdict(list)
    for lens in lenses:
        if lens[-1] == "-":
            label = lens[:-1]
            box = boxes[hash_string(label)]
            if (index := box_index(box, label)) != -1:
                # Found a box, remove it
                box.pop(index)
        else:
            label, value = lens.split("=")
            value = int(value)
            box = boxes[hash_string(label)]
            if (index := box_index(box, label)) != -1:
                # Found a box, replace the value
                box[index] = (label, value)
            else:
                box.append((label, value))
    return boxes


def focusing_power(boxes):
    result = 0
    for box in boxes:
        for slot, lens in enumerate(boxes[box], 1):
            result += (box + 1) * slot * lens[1]
    return result


lenses = data(15, sep=",")
print(f"Part 1: {sum(hash_string(string) for string in lenses)}")  # 510801
print(f"Part 2: {focusing_power(boxes(lenses))}")  # 212763

Part 1: 510801
Part 2: 212763


In [1218]:
# Day 16: The Floor Will Be Lava
def create_grid(lines):
    grid = {}
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            grid[row + 1j * col] = char
    return grid


def step_one_beam(grid, beam):
    "Return a list of new beams."
    # Directions: 1 = South, -1 = North, 1j = East, -1j = West
    pos, dir = beam
    # print(f"{grid[pos]} {pos} {dir=}")
    match grid[pos], dir:
        # Move one step in current dir, don't change dir
        case ".", _:
            return [(pos + dir, dir)]
        case "|", (-1 | 1):
            return [(pos + dir, dir)]
        case "-", (-1j | 1j):
            return [(pos + dir, dir)]
        # Mirror, horizontal movement
        case "/", (-1j | 1j):
            dir = dir * 1j
            return [(pos + dir, dir)]
        # Mirror, vertical movement
        case "/", (-1 | 1):
            dir = dir * -1j
            return [(pos + dir, dir)]
        # Mirror, horizontal movement
        case "\\", (-1j | 1j):
            dir = dir * -1j
            return [(pos + dir, dir)]
        # Mirror, vertical movement
        case "\\", (-1 | 1):
            dir = dir * 1j
            return [(pos + dir, dir)]
        # Split into two vertical beams
        case "|", (-1j | 1j):
            dir1, dir2 = -1, 1
            return [(pos + dir1, dir1), (pos + dir2, dir2)]
        # Split into two horizontal beams
        case "-", (-1 | 1):
            dir1, dir2 = -1j, 1j
            return [(pos + dir1, dir1), (pos + dir2, dir2)]
        case _:
            print(f"Unhandled:  {grid[pos]=} {dir=}")
            return (pos + dir, dir), None


def step_all_beams(grid, beams):
    new_beams = set()
    for beam in beams:
        new_beams.update(step_one_beam(grid, beam))
    # Only return beams that are inside grid
    return {beam for beam in new_beams if beam[0] in grid}


def energized(grid, start_beam=(0, 1j)):
    beams = {start_beam}
    all_visited = {start_beam[0]}
    visited = 1
    for i in range(10_000):
        beams = step_all_beams(grid, beams)
        all_visited.update({beam[0] for beam in beams})
        if i % 10 == 0:
            if visited == len(all_visited):
                return len(all_visited)
            visited = len(all_visited)
    return -1


def max_energized(grid):
    side_length = int(sqrt(len(grid)))  # Quadratic grid
    top_side = [(0 + col * 1j, 1) for col in range(side_length)]
    bottom_side = [(side_length - 1 + col * 1j, -1) for col in range(side_length)]
    left_side = [(row, 1j) for row in range(side_length)]
    right_side = [(row + (side_length - 1) * 1j, -1j) for row in range(side_length)]
    sides = top_side + bottom_side + left_side + right_side

    return max(energized(grid, beam) for beam in sides)


lines = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....""".splitlines()

lines = data(16)
grid = create_grid(lines)
print(f"Part 1: {energized(grid)}")  # 6883, 1s
print(f"Part 2: {max_energized(grid)}") # 7228, 2m 16s

Part 1: 6883
Part 2: 7228
